<a href="https://colab.research.google.com/github/Sai0472003/PatentLitigation_Impact/blob/main/Data_Cleaning2(FJC).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Causal effect of Patent Litigation Outcomes on Firm Financials


In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [37]:
JFC_1988_Present = pd.read_csv('/content/drive/MyDrive/ECO225_Final_Project/PART1/Civil_1988_to_Present.txt', sep='\t', encoding='latin1')

<ipython-input-37-caae5146eea8>:1: DtypeWarning: Columns (1,2,9,10,11,13,19,20,23,24,26,38,39,40,41,43,44) have mixed types. Specify dtype option on import or set low_memory=False.
  JFC_1988_Present = pd.read_csv('/content/drive/MyDrive/ECO225_Final_Project/PART1/Civil_1988_to_Present.txt', sep='\t', encoding='latin1')


In [67]:
cases = pd.read_csv('/content/drive/MyDrive/ECO225_Final_Project/PART1/cases.csv')
cases = cases[['case_row_id', 'case_number', 'case_name', 'case_cause', 'date_filed', 'date_closed', 'jurisdictional_basis']]

In [109]:
#ISOLATING PANTENT AND TRADENARK INFRINGEMENT SUITES, AND ONLY ORIGINAL PROCEEDINGS
Patents_1988_forwards = pd.DataFrame(JFC_1988_Present[JFC_1988_Present['NOS'].isin([830, 840])]) #only patent/trademark cases
#Patents_1988_forwards  = Patents_1988_forwards[Patents_1988_forwards ['ORIGIN'] == 1] #only original filings
#Patents_1988_forwards  = Patents_1988_forwards[Patents_1988_forwards ['ARBIT'].astype(str) == '-8'] #no formal arbitration courts

clean_JFC_patents = Patents_1988_forwards.copy()

#removing columns that aren't relavent or mostly empty
clean_JFC_patents.drop(columns=['ORIGIN','TITL', 'SECTION', 'SUBSECT', 'ARBIT',
                                'FILEJUDG', 'FILEMAG', 'MDLDOCK',  'TRANSDAT',
                                'TRANSOFF', 'TRANSDOC', 'TRANSORG', 'TDATEUSE',
                                'TRCLACT', 'TERMJUDG', 'TERMMAG', 'DJOINED',
                                'PRETRIAL', 'TRIBEGAN', 'TRIALEND', 'AMTREC',
                                'TRMARB', 'RESIDENC', 'PROSE', 'IFP', 'TAPEYEAR'],
                                 inplace=True)

In [111]:
clean_JFC_patents.loc[clean_JFC_patents['CLASSACT'] == -8, 'CLASSACT'] = 0

In [112]:
clean_JFC_patents['unique_identifier'] = clean_JFC_patents['DOCKET'].astype(str).str[:2] + "-cv-" + clean_JFC_patents['DOCKET'].astype(str).str[2:]
clean_JFC_patents['unique_identifier'] = clean_JFC_patents['JURIS'].astype(str) + "-" + clean_JFC_patents['FILEDATE'].astype(str).str.split("/").str[0] + "/" + clean_JFC_patents['FILEDATE'].astype(str).str.split("/").str[1] + "/" + clean_JFC_patents['unique_identifier'].astype(str)
clean_JFC_patents['unique_identifier'] = clean_JFC_patents['unique_identifier'] + "-" + clean_JFC_patents['TERMDATE'].astype(str)

#creating common unique identifier to match JFC dataset
cases['unique_identifier'] = cases['case_number'].astype(str).str.split(":", n=1).str[-1]
cases['unique_identifier'] = cases['date_filed'].astype(str).str.split("-").str[1] + "/" + cases['date_filed'].astype(str).str.split("-").str[2] + "/" +cases['unique_identifier'].astype(str)

for index, row in cases.iterrows():
    if row['jurisdictional_basis'] == "Federal Question":
        cases.loc[index, 'unique_identifier'] = "3-" + row['unique_identifier']
    elif row['jurisdictional_basis'] == "U.S. Government Plaintiff":
        cases.loc[index, 'unique_identifier'] = "1-" + row['unique_identifier']
    elif row['jurisdictional_basis'] == "U.S. Government Defendant":
        cases.loc[index, 'unique_identifier'] = "2-" + row['unique_identifier']
    elif row['jurisdictional_basis'] == "Diversity":
        cases.loc[index, 'unique_identifier'] = "4-" + row['unique_identifier']

cases['unique_identifier'] = cases['unique_identifier'] + "-" + cases['date_closed'].astype(str).str.split("-").str[1] + "/" + cases['date_closed'].astype(str).str.split("-").str[2] + "/" + cases['date_closed'].astype(str).str.split("-").str[0]

In [124]:
clean_JFC_patents.head()

,CIRCUIT,DISTRICT,OFFICE,DOCKET,FILEDATE,FDATEUSE,JURIS,NOS,JURY,CLASSACT,...,COUNTY,PLT,DEF,TERMDATE,PROCPROG,DISP,NOJ,JUDGMENT,STATUSCD,unique_identifier
42,0,90,1,8203478,12/08/1982,12/01/1982,3,830,-8,0,...,99999,CHUBB INTEG,NATL BANK WASH,02/10/1988,3,13,-8,-8,NaN,3-12/08/82-cv-03478-02/10/1988
47,0,90,1,8300001,01/03/1983,01/01/1983,3,830,-8,0,...,88888,NEWMAN,MOSSINGHOFF,02/17/1988,8,9,0,2,NaN,3-01/03/83-cv-00001-02/17/1988
188,0,90,1,8403233,10/22/1984,10/01/1984,3,830,-8,0,...,88888,CORP.COMMUNICATIONS,CAPITOL,06/01/1988,2,0,-8,-8,NaN,3-10/22/84-cv-03233-06/01/1988
241,0,90,1,8403720,12/06/1984,12/01/1984,3,840,-8,0,...,11001,WASH CHIROPTC HLTH,CHIROPTC INC,10/01/1987,3,3,-8,-8,NaN,3-12/06/84-cv-03720-10/01/1987
269,0,90,1,8500006,05/22/1985,05/01/1985,3,830,-8,0,...,0,FOUR PILLARS ENTP CO,CHANG ETAL,06/01/1988,4,6,0,2,NaN,3-05/22/85-cv-00006-06/01/1988


In [114]:
clean_JFC_patents.columns

Index(['CIRCUIT', 'DISTRICT', 'OFFICE', 'DOCKET', 'FILEDATE', 'FDATEUSE',
       'JURIS', 'NOS', 'JURY', 'CLASSACT', 'DEMANDED', 'COUNTY', 'PLT', 'DEF',
       'TERMDATE', 'PROCPROG', 'DISP', 'NOJ', 'JUDGMENT', 'STATUSCD',
       'unique_identifier'],
      dtype='object')

In [115]:
cases.head()

,case_row_id,case_number,case_name,case_cause,date_filed,date_closed,jurisdictional_basis,unique_identifier
0,78028.0,4:92-cv-00055,"Melba Creek Mining, Inc. et al v. Sykes et al",No cause code entered,1992-12-22,1993-02-16,Federal Question,3-12/22/92-cv-00055-02/16/1993
1,75332.0,2:10-cv-00601,Bowhall v. Johnson & Johnson Inc. et al (MAG+),15:1126 Patent Infringement,2010-07-14,2010-10-12,Federal Question,3-07/14/10-cv-00601-10/12/2010
2,6908.0,1:98-cv-00596,Mettke v. Touchnet Information,35:183 Patent Infringement,1998-03-16,1998-09-21,Federal Question,3-03/16/98-cv-00596-09/21/1998
3,6933.0,1:98-cv-00654,"Monsanto Co. Inc. v. Styron, et al",28:1338 Patent Infringement,1998-06-29,1999-03-15,Federal Question,3-06/29/98-cv-00654-03/15/1999
4,86730.0,1:18-cv-00012,Sundesa LLC v. Eurark LLC,35:271 Patent Infringement,2018-02-13,2018-07-06,Federal Question,3-02/13/18-cv-00012-07/06/2018


In [129]:
merged = pd.merge(cases, clean_JFC_patents, on='unique_identifier', how='inner')

In [130]:
merged.groupby('case_number')['unique_identifier'].nunique().value_counts()

,count
unique_identifier,
1,54512
2,3420
3,275
4,17


In [131]:
valid_cases = merged.groupby('case_number')['unique_identifier'].nunique() == 1

# Filter the dataframe to keep only these valid case_numbers
merged = merged[merged['case_number'].isin(valid_cases[valid_cases].index)]
merged.drop_duplicates(inplace=True)

In [132]:
merged['case_number'].nunique()

54512

In [133]:
merged

,case_row_id,case_number,case_name,case_cause,date_filed,date_closed,jurisdictional_basis,unique_identifier,CIRCUIT,DISTRICT,...,DEMANDED,COUNTY,PLT,DEF,TERMDATE,PROCPROG,DISP,NOJ,JUDGMENT,STATUSCD
1,6908.0,1:98-cv-00596,Mettke v. Touchnet Information,35:183 Patent Infringement,1998-03-16,1998-09-21,Federal Question,3-03/16/98-cv-00596-09/21/1998,11,26,...,0,1015,METTKE,TOUCHNET INFORMATION,09/21/1998,3,14,-8,-8,NaN
2,6933.0,1:98-cv-00654,"Monsanto Co. Inc. v. Styron, et al",28:1338 Patent Infringement,1998-06-29,1999-03-15,Federal Question,3-06/29/98-cv-00654-03/15/1999,11,28,...,0,1097,MONSANTO CO. INC.,"STYRON, ET AL",03/15/1999,10,13,-8,-8,NaN
3,86730.0,1:18-cv-00012,Sundesa LLC v. Eurark LLC,35:271 Patent Infringement,2018-02-13,2018-07-06,Federal Question,3-02/13/18-cv-00012-07/06/2018,8,60,...,0,88888,SUNDESA LLC,EURARK LLC,07/06/2018,3,13,0,0,L
4,30787.0,2:99-cv-00674,"Star Video Duplictns v. US Magnetics Corp, et al",15:1125 Trademark Infringement (Lanham Act),1999-04-09,1999-12-16,Federal Question,3-04/09/99-cv-00674-12/16/1999,9,70,...,0,4013,STAR VIDEO DUPLICTNS,"US MAGNETICS CORP, ET AL",12/16/1999,10,13,-8,-8,NaN
5,30985.0,2:99-cv-02348,"Brother Industries v. Jing Mold Enterprise, et al",28:1338 Patent Infringement,1999-03-05,2001-03-13,Federal Question,3-03/05/99-cv-02348-03/13/2001,9,73,...,0,99999,BROTHER INDUSTRIES,"JING MOLD ENTERPRISE, ET AL",03/13/2001,3,13,-8,-8,L
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62258,30718.0,2:99-cv-00216,"Impact Roller Tech, et al v. Star Aggregates Inc",28:1338 Patent Infringement,1999-10-06,2000-03-28,Federal Question,3-10/06/99-cv-00216-03/28/2000,10,89,...,0,88888,"IMPACT ROLLER TECH, ET AL",STAR AGGREGATES INC,03/28/2000,5,5,0,1,NaN
62261,28107.0,2:93-cv-00004,Construction Tech v. Heating,35:271 Patent Infringement,1993-01-04,1993-08-11,Federal Question,3-01/04/93-cv-00004-08/11/1993,10,89,...,0,88888,CONSTRUCTION TECH,HEATING,08/11/1993,3,12,-8,-8,NaN
62263,28883.0,2:95-cv-00217,Wyoming Woolens v. Acorn Products Co,28:1338 Patent Infringement,1995-09-29,1995-10-06,Federal Question,3-09/29/95-cv-00217-10/06/1995,10,89,...,0,56039,WYOMING WOOLENS,ACORN PRODUCTS CO,10/06/1995,1,12,-8,-8,NaN
62265,29713.0,2:97-cv-00019,Chey Land & Cattle v. Crown Cork & Seal Co,35:271 Patent Infringement,1997-01-24,1997-07-25,Federal Question,3-01/24/97-cv-00019-07/25/1997,10,89,...,2000,56021,CHEY LAND & CATTLE,CROWN CORK & SEAL CO,07/25/1997,1,12,-8,-8,NaN


In [191]:
import wrds
db = wrds.Connection(wrds_username="sai047")

Loading library list...
Done


In [192]:
query = """
SELECT gvkey, conm
FROM comp.company
"""
comp_names = db.raw_sql(query)

In [193]:
comp_names

,gvkey,conm
0,001000,A & E PLASTIK PAK INC
1,001001,A & M FOOD SERVICES INC
2,001002,AAI CORP
3,001003,A.A. IMPORTING CO INC
4,001004,AAR CORP
...,...,...
55007,356128,JOINT STOCK COMPANY KASPI KZ
55008,356289,SOLARBANK CORP
55009,356687,ATHA ENERGY CORP
55010,356859,ACUREN CORP


In [194]:
gvkey_plt = pd.DataFrame(merged[['PLT']])
gvkey_def = pd.DataFrame(merged[['DEF']])

In [195]:
def clean_names(name):
    name = name.replace(",", '')  # Remove Commas
    name = name.replace(".", "")  # Remove periods
    name = name.replace("ET AL", "").strip()  # Remove 'ET AL' (case insensitive if already upper)
    return name

gvkey_def = pd.DataFrame(gvkey_def['DEF'].apply(clean_names)) # Apply to the 'DEF' column
gvkey_plt = pd.DataFrame(gvkey_plt['PLT'].apply(clean_names)) # Apply to the 'PLT' column
comp_names['conm'].apply(clean_names)
merged['PLT'] = merged['PLT'].apply(clean_names)
merged['DEF'] = merged['DEF'].apply(clean_names)

In [196]:
comp_names['name'] = comp_names['conm']
comp_names.drop(columns=['conm'], inplace=True)
gvkey_plt['name'] = gvkey_plt['PLT']
gvkey_def['name'] = gvkey_def['DEF']

In [197]:
gvkey_def = gvkey_def.merge(comp_names, on='name', how='inner')
gvkey_def.drop(columns=['name'], inplace=True)

gvkey_plt = gvkey_plt.merge(comp_names, on='name', how='inner')
gvkey_plt.drop(columns=['name'], inplace=True)

In [198]:
gvkey_def

,DEF,gvkey
0,INTERMEDICS INC,006045
1,TEXAS INSTRUMENTS INC,010499
2,ENCORE COMPUTER CORP,004344
3,EBAY INC,114524
4,LEGGETT & PLATT INC,006649
...,...,...
6027,OEC MEDICAL SYSTEMS INC,003935
6028,CRAY INC,061350
6029,CISCO SYSTEMS INC,020779
6030,HUBBELL INC,005764


In [210]:
gvkey_def.groupby('DEF')['gvkey'].nunique().value_counts()

,count
gvkey,
1,1747


In [199]:
gvkey_plt

,PLT,gvkey
0,KIDDE INC,006430
1,SONOCO PRODUCTS CO,009815
2,DCL INC,003688
3,INTERGRAPH CORP,006036
4,ALABAMA POWER CO,001225
...,...,...
2650,GILEAD SCIENCES INC,024856
2651,PFIZER INC,008530
2652,JAPAN TOBACCO INC,223528
2653,PFIZER INC,008530


In [220]:
merged.columns

Index(['case_row_id', 'case_number', 'case_name', 'case_cause', 'date_filed',
       'date_closed', 'jurisdictional_basis', 'unique_identifier', 'CIRCUIT',
       'DISTRICT', 'OFFICE', 'DOCKET', 'FILEDATE', 'FDATEUSE', 'JURIS', 'NOS',
       'JURY', 'CLASSACT', 'DEMANDED', 'COUNTY', 'PLT', 'DEF', 'TERMDATE',
       'PROCPROG', 'DISP', 'NOJ', 'JUDGMENT', 'STATUSCD'],
      dtype='object')

In [221]:
plaintiff_RnD = pd.merge(merged[['case_row_id', 'case_number', 'date_filed',
                                 'date_closed', 'PLT']], gvkey_plt, on='PLT',
                                  how='inner')
plaintiff_RnD['case_number'].nunique()
plaintiff_RnD.drop_duplicates(inplace=True)

In [222]:
plaintiff_RnD.head()

,case_row_id,case_number,date_filed,date_closed,PLT,gvkey
0,58881.0,4:85-cv-00399,1985-04-25,1989-10-23,KIDDE INC,006430
1,59800.0,4:96-cv-02682,1996-09-03,1999-02-23,SONOCO PRODUCTS CO,009815
6,7660.0,1:99-cv-01028,1999-04-26,1999-06-08,DCL INC,003688
7,64094.0,5:97-cv-03023,1997-11-17,2002-05-06,INTERGRAPH CORP,006036
8,30252.0,2:98-cv-00323,1998-02-11,1998-06-10,ALABAMA POWER CO,001225


In [226]:
defendent_RnD = pd.merge(merged[['case_row_id', 'case_number', 'date_filed',
                                 'date_closed', 'DEF']], gvkey_def, on='DEF',
                                  how='inner')

defendent_RnD['case_number'].nunique()
defendent_RnD.drop_duplicates(inplace=True)

In [227]:
defendent_RnD.head()

,case_row_id,case_number,date_filed,date_closed,DEF,gvkey
0,59316.0,4:93-cv-00626,1993-06-25,1996-10-08,INTERMEDICS INC,006045
5,78121.0,2:11-cv-01037,2011-12-06,2015-03-30,TEXAS INSTRUMENTS INC,010499
25,78037.0,5:92-cv-02573,1992-11-02,1996-07-29,ENCORE COMPUTER CORP,004344
26,26100.0,1:15-cv-01059,2015-06-24,2017-05-15,EBAY INC,114524
69,66037.0,5:11-cv-01349,2011-04-20,2012-09-17,LEGGETT & PLATT INC,006649


In [218]:
defendent_RnD.columns

Index(['case_row_id', 'case_number', 'case_name', 'case_cause', 'date_filed',
       'date_closed', 'jurisdictional_basis', 'unique_identifier', 'CIRCUIT',
       'DISTRICT', 'OFFICE', 'DOCKET', 'FILEDATE', 'FDATEUSE', 'JURIS', 'NOS',
       'JURY', 'CLASSACT', 'DEMANDED', 'COUNTY', 'PLT', 'DEF', 'TERMDATE',
       'PROCPROG', 'DISP', 'NOJ', 'JUDGMENT', 'STATUSCD', 'gvkey'],
      dtype='object')